In [1]:
#数据读取 为了方便大家更好的理解数据，我们重命名为中文
import numpy  as np
import pandas as pd
import os
data      = pd.read_csv("账户交易信息.csv", encoding="gbk")
train     = pd.read_csv("训练集标签.csv", encoding="gbk")
user_info = pd.read_csv("账户静态信息.csv", encoding="utf-8")
test      = pd.read_csv("test_dataset.csv", encoding="utf-8")
data = data.rename({
                    'jylsxh': '交易流水序号',
                    'zhdh': '账户代号',
                    'dfzh': '对方账号',
                    'jdbz': '借贷标志',
                    'jyje': '交易金额',
                    'jyye': '交易余额',
                    'dfhh': '对方行号',
                    'jyrq': '交易日期',
                    'jysj': '交易时间',
                    'jyqd': '交易渠道',
                    'zydh': '摘要代号',
                    'jdbj': '借贷标记',
                    'zhye': '账户余额',
                    'dfmccd': '对方名称长度'}, axis=1
                  )

train    = train.rename({'zhdh': '账户代号', 
                         'black_flag': '标签'}, axis=1
                       )

user_info =  user_info.rename({'zhdh': '账户代号',
                               'khrq':'开户日期', 
                               'xb': '性别', 
                             'khjgdh':'开户机构代号'}, axis=1
                              )

test    =  test.rename({'zhdh': '账户代号'},axis=1)

In [2]:
# 匹配上属性数据
df = pd.merge(data, user_info, on='账户代号', how='left')
df = pd.merge(df, train, on='账户代号', how='left')
df.head()


import warnings
import pandas as pd
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)


# 异构网络构建
df1 = df[['账户代号','对方账号']]

df1['账户代号'] = 'zh_' +df1[['账户代号']]
df1['对方账号'] = 'df_' +df1[['对方账号']]

df1.columns = ['账户代号','对方账号']


df2 = df[['账户代号','交易渠道']]
df2['账户代号'] = 'zh_' +df2[['账户代号']]
df2['交易渠道'] = 'qd_' +df2[['交易渠道']]
df2.columns = ['账户代号','对方账号']
 
    
df3 = df[['账户代号','摘要代号']]
df3['账户代号'] = 'zh_' +df3[['账户代号']]
df3['摘要代号'] = 'zy_' +df3[['摘要代号']]
df3.columns = ['账户代号','对方账号']
 
df4 = df[['账户代号','对方行号']]
df4['账户代号'] = 'zh_' +df4[['账户代号']]
df4['对方行号'] = 'hh_' +df4[['对方行号']]
df4.columns = ['账户代号','对方账号']
 

df5 = df[['对方账号','交易渠道']]
df5['对方账号'] = 'df_' +df5[['对方账号']]
df5['交易渠道'] = 'qd_' +df5[['交易渠道']]
df5.columns = ['账户代号','对方账号']
 
    
df6 = df[['对方账号','摘要代号']]
df6['对方账号'] = 'df_' +df6[['对方账号']]
df6['摘要代号'] = 'zy_' +df6[['摘要代号']]
df6.columns =['账户代号','对方账号']
 
df7 = df[['对方账号','对方行号']]
df7['对方账号'] = 'df_' +df7[['对方账号']]
df7['对方行号'] = 'hh_' +df7[['对方行号']]
df7.columns = ['账户代号','对方账号']
 
df8 = df[['交易渠道','摘要代号']]
df8['交易渠道'] = 'qd_' +df8[['交易渠道']]
df8['摘要代号'] = 'zy_' +df8[['摘要代号']]
df8.columns = ['账户代号','对方账号']
 
df9 = df[['交易渠道','对方行号']]
df9['交易渠道'] = 'qd_' +df9[['交易渠道']]
df9['对方行号'] = 'hh_' +df9[['对方行号']]
df9.columns =['账户代号','对方账号']
 

df10 = df[['摘要代号','对方行号']]
df10['摘要代号'] = 'zy_' +df10[['摘要代号']]
df10['对方行号'] = 'hh_' +df10[['对方行号']]
df10.columns =['账户代号','对方账号']
 

# 四个节点、三种关系合并
dda = pd.concat([df1,df2,df3,df4,df5,df6,df7,df8,df9,df10])


# 对数据进行聚合
dda = dda.groupby(['账户代号','对方账号']).agg({'对方账号': ['count']}).reset_index()
dda.columns=['账户代号','对方账号','CNT']

# 剔除自己指向自己的节点
print('剔除前的数据:\n',dda.shape)

dda = dda[dda['账户代号']!=dda['对方账号']]
print('剔除后的数据:\n',dda.shape)


剔除前的数据:
 (767720, 3)
剔除后的数据:
 (767720, 3)


In [3]:
# 算法分群模块
import networkx as nx
import matplotlib.pyplot as plt

# 常规-图数据转换
da = dda[['账户代号','对方账号']].values
G  = nx.Graph()
for num in range(len(da)):
    G.add_edge(str(da[num,0]),str(da[num,1]))

In [4]:
#极大连通子图算法
print('极大连通子图算法开始...')
com = list(nx.connected_components(G))
#格式整理
df_com_connected  = pd.DataFrame()
for i in range(0, len(com)):
    d = pd.DataFrame({'Group_id': [i] * len(com[i]), '账户代号': list(com[i])})
    df_com_connected = pd.concat([df_com_connected,d])


极大连通子图算法开始...


In [6]:

####################################################################
#LPA 算法
print('LPA算法开始...')
from networkx.algorithms.community import asyn_lpa_communities as lpa
com = list(lpa(G))
#格式整理
df_com_lpa = pd.DataFrame()
for i in range(0,len(com)):
    d = pd.DataFrame({'Group_id':[i]*len(com[i]),'账户代号':list(com[i])})
    df_com_lpa = pd.concat([df_com_lpa,d])



LPA算法开始...


In [8]:
####################################################################
#Louvain算法
print('Louvain算法...')
from community import community_louvain
com = community_louvain.best_partition(G,resolution=1.5)
#格式整理
df_com_louvain = pd.DataFrame()
df_com_louvain = pd.DataFrame({'Group_id':com.values(),
                               '账户代号':com.keys()})
print('计算结束')

Louvain算法...
计算结束


In [11]:
# 社群数据画像
'''
pandasql 可以把SQL代码写到这里，有些复杂的逻辑方便SQL写，
并且日常应用也好参考,
没有的安装下 pip install pandasql

# 交替看看三个算法的情况
df_com_connected
df_com_lpa
df_com_louvain

'''
# 训练集-测试集也做同样的处理
train['账户代号'] = 'zh_'+train['账户代号']
test['账户代号']  = 'zh_'+test['账户代号']


from   pandasql import sqldf
query = '''
select 
group_id,
all_cnt,
test_cnt,
train_0_cnt,
train_1_cnt,
round(cast(train_1_cnt as double)/(train_0_cnt+train_1_cnt),4) as rate
from
(select
a.group_id,
count(a.账户代号)                        as all_cnt,
sum(case when 标签=0 then 1 else 0 end) as train_0_cnt,
sum(case when 标签=1 then 1 else 0 end) as train_1_cnt,
sum(case when c.账户代号 is not null then 1 else 0 end) as test_cnt
from
(select
group_id,
账户代号
from df_com_louvain -- 替换算法结果
) a
left join
(select
账户代号,
标签
from train
) b
on a.账户代号 = b.账户代号
left join
(select 
账户代号
from test
) c
on a.账户代号 = c.账户代号
group by group_id
) a
order by train_1_cnt desc
limit 2000
'''

sqldf(query).head(50)


,group_id,all_cnt,test_cnt,train_0_cnt,train_1_cnt,rate
0,2,33453,285,12,67,0.8481
1,14,3597,1698,330,61,0.1560
2,4,23266,448,65,31,0.3229
3,7,2110,93,1,19,0.9500
4,1,4014,80,13,16,0.5517
5,8,3301,157,23,14,0.3784
6,6,11933,889,247,11,0.0426
7,3,12208,238,55,10,0.1538
8,15,1482,54,8,9,0.5294
9,11,3810,102,14,8,0.3636


In [12]:
# 分群结果概览

'''
团伙用户数统计
df_com_connected
df_com_lpa
df_com_louvain
'''

df_com_louvain.groupby('Group_id').count().sort_values(by = '账户代号',ascending=False).head(12)


,账户代号
Group_id,
2,33453
4,23266
0,22514
3,12208
6,11933
1,4014
11,3810
14,3597
8,3301


In [13]:
# 训练集-测试集也做同样的处理
train['账户代号'] = 'zh_'+train['账户代号']
test['账户代号']  = 'zh_'+test['账户代号']


In [14]:
# 交替看看三个算法的情况
'''
df_com_connected
df_com_lpa
df_com_louvain
'''

query = '''
select 
a.账户代号,
a.对方账号
from
(select 
账户代号,
对方账号
from dda
)a

left join
(select
group_id,
账户代号
from df_com_louvain  --替换算法查看
where group_id=4
) b 
on a.账户代号 = b.账户代号

left join
(select
group_id,
账户代号
from df_com_louvain --替换算法查看
where group_id=4
) c 
on a.对方账号 = c.账户代号
where b.账户代号 is not null or c.账户代号 is not null -- 用or 或者 and 可以调整查看
'''
dg = sqldf(query)

df_u_g = pd.DataFrame(pd.concat([dg['账户代号'],dg['对方账号']]))
df_u_g.columns = ['账户代号']


#节点来源处理
query = '''
select 
a.账户代号,
case when b.账户代号 is not null and 标签=1 then 'is_black_1' 
     when b.账户代号 is not null and 标签=0 then 'is_black_0' 
     when c.账户代号 is not null then 'is_test' else 'is_other' end  as flag
from
(select
账户代号
from df_u_g
group by 账户代号
) a
left join
(select
账户代号,
标签
from train
) b
on a.账户代号 = b.账户代号
left join
(select 
账户代号
from test
) c
on a.账户代号 = c.账户代号
'''

flag = sqldf(query)


# 社区还原并可视化
d = dg[['账户代号','对方账号']].values
G  = nx.Graph()
for num in range(len(d)):
    G.add_edge(str(d[num,0]),str(d[num,1]))

#显示图片
plt.figure(figsize=(5,5),dpi=1200)


# 颜色设置
'''
训练集-黑样本  -- 红色 r
训练集-白羊本  -- 黑色 black
测试集 --------- 橘色 Orange
对手账户-------- 灰色
'''
colors_dic={'is_black_1':'r','is_black_0':'black','is_test':'Orange','is_other':'#9AC0CD'}
flag['colors'] = flag['flag'].map(colors_dic)


#节点大小设置，与度关联
node_size = [G.degree(i)**0.75*1 for i in G.nodes()]
node_color = [dict(zip(flag['账户代号'].tolist(), flag['colors'].tolist()))[i] for i in G.nodes()]


#kamada_kawai_layout spring_layout
nx.draw_networkx(G,
                 pos = nx.spring_layout(G),
                 node_color = node_color,
                 edge_color = '#2E8B57',
                 with_labels=False,
                 #font_size = 2,
                 node_size = node_size,
                 alpha = 0.98,
                 width = 0.03
                 )
plt.axis('off')
plt.show()